In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from mdt_agent import tools

In [4]:
import os
os.environ['MY_DAILY_TASKS_URL'] = 'http://localhost:3000'

In [5]:
base_url = os.getenv('MY_DAILY_TASKS_URL')
todo_tools = tools.TodoTools(base_url=base_url)